## 0. Important imports

#### token hugging face
This token is mandatory. Register on the website and generate one.

In [1]:
access_token = "secret"

In [2]:
from huggingface_hub import login
login(token=access_token)

/miniconda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### 1. Importing libraries

In [3]:
import torch

In [4]:
GPU_use = 2
st = "cuda:"+str(GPU_use)
print(torch.cuda.current_device())
torch.cuda.set_device(GPU_use) 
print(torch.cuda.current_device())
#st = "cpu"
#torch.device(st)

0
2


### 2. Quick look to the dataset updated before

In [5]:
import datasets

In [8]:
ds = datasets.load_dataset('username/dataset_name')

In [9]:
ds

DatasetDict({
    train: Dataset({
        features: ['context', 'question', 'answer'],
        num_rows: 58800
    })
})

In [10]:
trn = ds['train']
import json
print(json.dumps(trn[14], indent=2))

{
  "context": "table([ eof x: -69 y: -148 z: 662, gripper: open , Black-Cup x: -276 y: -137 z: 80, Plate x: -47 y: 251 z: 30, Bowl x: -219 y: 41 z: 100, White-Cup x: -195 y: 311 z: 80, Apple x: 131 y: 14 z: 50 or: 124, Banana x: -51 y: 118 z: 30 or: 99, Carrot x: 238 y: 13 z: 30 or: 116, Chili x: 322 y: 179 z: 40 or: 143, Coke-Can x: -12 y: 85 z: 70 or: 6, Corn x: -376 y: -282 z: 30 or: 9, Eggplant x: -300 y: 16 z: 50 or: 165, Gear x: -347 y: -425 z: 20 or: 70, Grape x: 60 y: -106 z: 20 or: 81, Green-Block x: 119 y: -328 z: 80 or: 76, Green-Cylinder x: 115 y: -126 z: 40 or: 151, Green-Parallelepiped x: -20 y: -382 z: 80 or: 18, Green-Pepper x: 418 y: 207 z: 80 or: 100, Grey-Block x: -161 y: 248 z: 80 or: 86, Orange x: -149 y: 394 z: 50 or: 104, Pineapple x: -158 y: -276 z: 50 or: 170, Red-Block x: 201 y: 176 z: 40 or: 138, Red-Triangle x: 421 y: -18 z: 40 or: 106, Rivella Bottle x: -222 y: -439 z: 100 or: 169, Tomato x: 361 y: 199 z: 50 or: 147, Yellow Ball x: 251 y: 35 z: 50 or: 57 ]

### 3.Merging the fine-tuned model with the GPT

In [11]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoModel, TFBertForQuestionAnswering,TFAutoModelWithLMHead

In [ ]:
base_model = 'meta-llama/Meta-Llama-3-8B'
tokr = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.bfloat16, device_map=GPU_use,token=access_token)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

In [ ]:
qlora_model = './../qlora-out'

In [ ]:
model = PeftModel.from_pretrained(model, qlora_model,device_map=GPU_use,token=access_token)
model = model.merge_and_unload()

### 4. Saving the model and push to the hub

In [ ]:
model.save_pretrained('PEFT-trained-model')

In [ ]:
# push to the hub
model.push_to_hub("marcomaccarini/TEST_LAB")

### 5. Testing the model

In [ ]:
fmt = """
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
USER: {}
===
{}
ASSISTANT:"""

def sql_prompt(d): return fmt.format(d["context"], d["question"])
def question(table, quest):
    tst = dict(**trn[8])
    tst['context'] = table
    tst['question'] = quest
    return sql_prompt(tst)

In [ ]:
%%time
t = 'table([ eof x: -69 y: -148 z: 662, gripper: open , apple x: 100, y: 30, z: 250; banana x: 300, y: 220, z: 120; pear x: 20, y: 200, z: 60; melon x: 230, y: 100, z: 50; cup x: 230, y: 320, z: 30;])'
q = 'pick apple and place to Cup'

test = question(t,q)
toks = tokr(test, return_tensors="pt")
res = model.generate(**toks.to(st), max_new_tokens=100, top_p = 0).to('cpu')

In [ ]:
#%%time
print(tokr.batch_decode(res)[0].replace("*","\n"))